<a href="https://colab.research.google.com/github/DenyT17/Football_Object_Detection/blob/main/Perspective_Transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
%cd "/content/gdrive/MyDrive/Football_Detection
!git clone https://github.com/FootballAnalysis/footballanalysis.git

In [7]:
%%writefile "/content/gdrive/MyDrive/Football_Detection/footballanalysis/Bird's eye view/main.py"

from elements.perspective_transform import Perspective_Transform
from elements.assets import transform_matrix, detect_color
from arguments import Arguments
from yolov5.utils.plots import plot_one_box
from ultralytics import YOLO

import torch
import os
import cv2
import numpy as np
import sys

def main(opt):
    classNames = ['ball', 'goalkeeper', 'player', 'referee']
    # Load models
    perspective_transform = Perspective_Transform()

    model = YOLO("/content/gdrive/MyDrive/Football_Detection/runs/detect/train2/weights/best.pt")
    # Video capture
    cap = cv2.VideoCapture(r"/content/gdrive/MyDrive/Football_Detection/video/finall.mp4")
    frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)

    w = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    h = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

    # Save output
    if opt.save:
        output_name = opt.source.split('/')[-1]
        output_name = output_name.split('.')[0] + '_output.' + output_name.split('.')[-1]

        output_path = os.path.join(os.getcwd(), 'inference/output')
        os.makedirs(output_path, exist_ok=True)
        output_name = os.path.join(os.getcwd(), 'inference/output', output_name)

        w = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
        h = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

        out = cv2.VideoWriter(output_name,  
                                cv2.VideoWriter_fourcc(*'mp4v'), 
                                opt.outputfps, (int(w), int(h)))


    frame_num = 0

    # Black Image (Soccer Field)
    bg_ratio = int(np.ceil(w/(3*115)))
    gt_img = cv2.imread('./inference/black.jpg')
    gt_img = cv2.resize(gt_img,(115*bg_ratio, 74*bg_ratio))
    gt_h, gt_w, _ = gt_img.shape


    while(cap.isOpened()):
        
        ret, frame = cap.read()
        bg_img = gt_img.copy()

        if ret:
            main_frame = frame.copy()
            yoloOutput = model(frame)

            # Output: Homography Matrix and Warped image 
            if frame_num % 5 ==0: # Calculate the homography matrix every 5 frames
                M, warped_image = perspective_transform.homography_matrix(main_frame)

            if yoloOutput:
                # The homography matrix is applied to the center of the lower side of the bbox.
                for r in yoloOutput:
                    boxes = r.boxes
                    for box in boxes:
                   
                      x1,y1,x2,y2 = box.xyxy[0]
                      x1,y1,x2,y2 = int(x1), int(y1), int(x2), int(y2)
                      x_center = x1+((x2 - x1)/2) 
                      y_center = y1+((y2 - y1)/2) 
                      
                      if classNames[int(box.cls[0])] == 'referee':
                          plot_one_box([x1,y1,x2,y2], frame, (102, 0, 102), label="referee")

                      elif classNames[int(box.cls[0])] == 'player':
                          plot_one_box([x1,y1,x2,y2], frame, (102, 0, 102), label="player")
                          coords = transform_matrix(M, (x_center, y_center), (h, w), (gt_h, gt_w))
                          try:

                              cv2.circle(bg_img, coords, bg_ratio + 1, (255,100,200), -1)
                          except:
                            pass
                      elif classNames[int(box.cls[0])] == 'ball':
                          coords = transform_matrix(M, (x_center, y_center), (h, w), (gt_h, gt_w))
                          cv2.circle(bg_img, coords, bg_ratio + 1, (102, 0, 102), -1)
                          plot_one_box([x1,y1,x2,y2], frame, (102, 0, 102), label="ball")

            frame[frame.shape[0]-bg_img.shape[0]:, frame.shape[1]-bg_img.shape[1]:] = bg_img  
            
            if opt.view:
                cv2.imshow('frame',frame)
                if cv2.waitKey(1) & ord('q') == 0xFF:
                    break

            # Saving the output
            if opt.save:
                out.write(frame)

            frame_num += 1
        else:
            break

        sys.stdout.write(
            "\r[Input Video : %s] [%d/%d Frames Processed]"
            % (
                opt.source,
                frame_num,
                frame_count,
            )
        )

    if opt.save:
        print(f'\n\nOutput video has been saved in {output_path}!')
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':

    opt = Arguments().parse()
    with torch.no_grad():
        main(opt)


Overwriting /content/gdrive/MyDrive/Football_Detection/footballanalysis/Bird's eye view/main.py


In [4]:
%cd "/content/gdrive/MyDrive/Football_Detection/footballanalysis/Bird's eye view"
!pip install -r requirements.txt
!pip install ultralytics

/content/gdrive/MyDrive/Football_Detection/footballanalysis/Bird's eye view
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.4/831.4 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.1/22.1 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 91.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyflann-py3: filename=pyflann_py3-0.1.0-py3-none-any.whl size=7186987 sha256=2c1368780debb59c818c1b0b9fd48ceaccfcd064479d2f4e7a0b0193ca37f037
  Stored in directory: /root/.cache/pip/wheels/d9/5f/aa/82122a3a88158eb3a45dd9d727baca078d58af3291e4a6a8ad
Successfully built pyflann-py3
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0+cu118
    Uninstalling torch-2.0.0+cu118:
      Successfully uninstalled torch-2.0.0+cu118
  Attempting uninstall: torchvision
    Found existing installation: torchvis

In [5]:
!python main.py --source "/content/gdrive/MyDrive/Football_Detection/video/finall.mp4" --save

Perspective Transform model loaded!

0: 384x640 1 goalkeeper, 18 players, 2 referees, 330.4ms
Speed: 0.8ms preprocess, 330.4ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)
[Input Video : /content/gdrive/MyDrive/Football_Detection/video/finall.mp4] [1/54 Frames Processed]
0: 384x640 1 goalkeeper, 18 players, 2 referees, 324.2ms
Speed: 0.9ms preprocess, 324.2ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)
[Input Video : /content/gdrive/MyDrive/Football_Detection/video/finall.mp4] [2/54 Frames Processed]
0: 384x640 1 goalkeeper, 19 players, 2 referees, 314.9ms
Speed: 0.6ms preprocess, 314.9ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)
[Input Video : /content/gdrive/MyDrive/Football_Detection/video/finall.mp4] [3/54 Frames Processed]
0: 384x640 1 goalkeeper, 20 players, 2 referees, 304.1ms
Speed: 0.7ms preprocess, 304.1ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)
[Input Video : /content/gdrive/MyDrive/Foot